In [4]:
import theano
import theano.tensor as T
import numpy as np

from lasagne.updates import sgd,apply_momentum

In [5]:
VOCAB = ['#', 'green','red','blue','square','circle','$']
Dic = {'#':0, 'green':1, 'red':2, 'blue':3, 'square':4, 'circle':5, '$':6}
ONE_HOT_VECS = np.eye(len(VOCAB))


In [6]:
from src.VGG_16 import *
#%matplotlib inline 




    

Using Theano backend.


In [7]:
def get_string(list_of_vec):
    str = " "
    for i in np.arange(len(list_of_vec)):
        str += " "+VOCAB[np.argmax(list_of_vec[i])]

    return str.strip()


In [8]:
red_squares, thumb_red_squares = load_images_from_folder("red_square")
red_square_label = np.asarray([(ONE_HOT_VECS[Dic["red"]]),(ONE_HOT_VECS[Dic["square"]])])
red_square_labels =  np.repeat([red_square_label], len(red_squares),axis=0)

green_squares, thumb_green_squares = load_images_from_folder("green_square")
green_square_label = np.asarray([(ONE_HOT_VECS[Dic["green"]]),(ONE_HOT_VECS[Dic["square"]])])
green_square_labels =  np.repeat([green_square_label], len(green_squares),axis=0)

blue_squares, thumb_blue_squares = load_images_from_folder("blue_square")
blue_square_label = np.asarray([(ONE_HOT_VECS[Dic["blue"]]),(ONE_HOT_VECS[Dic["square"]])])
blue_square_labels =  np.repeat([blue_square_label], len(blue_squares),axis=0)

red_circles, thumb_red_circles = load_images_from_folder("red_circle")
red_circle_label = np.asarray([(ONE_HOT_VECS[Dic["red"]]),(ONE_HOT_VECS[Dic["circle"]])])
red_circle_labels =  np.repeat([red_circle_label], len(red_circles),axis=0)

green_circles, thumb_green_circles = load_images_from_folder("green_circle")
green_circle_label = np.asarray([(ONE_HOT_VECS[Dic["green"]]),(ONE_HOT_VECS[Dic["circle"]])])
green_circle_labels =  np.repeat([green_circle_label], len(green_circles),axis=0)

blue_circles, thumb_blue_circles = load_images_from_folder("blue_circle")
blue_circle_label = np.asarray([(ONE_HOT_VECS[Dic["blue"]]),(ONE_HOT_VECS[Dic["circle"]])])
blue_circle_labels =  np.repeat([blue_circle_label], len(blue_circles),axis=0)



In [9]:
shapes = []
shapes.extend(red_squares)
shapes.extend(green_squares)
shapes.extend(blue_squares)
shapes.extend(red_circles)
shapes.extend(green_circles)
#shapes.extend(blue_circles)

labels = []
labels.extend(red_square_labels)
labels.extend(green_square_labels)
labels.extend(blue_square_labels)
labels.extend(red_circle_labels)
labels.extend(green_circle_labels)
#labels.extend(blue_circle_labels)

vgg_shapes = []
vgg_shapes.extend(red_squares)
vgg_shapes.extend(green_squares)
vgg_shapes.extend(blue_squares)
vgg_shapes.extend(red_circles)
vgg_shapes.extend(green_circles)
vgg_shapes.extend(blue_circles)

vgg_labels = []
vgg_labels.extend(red_square_labels)
vgg_labels.extend(green_square_labels)
vgg_labels.extend(blue_square_labels)
vgg_labels.extend(red_circle_labels)
vgg_labels.extend(green_circle_labels)
vgg_labels.extend(blue_circle_labels)


In [10]:
vgg = VGG_16('vgg16_weights.h5')
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
vgg.model.compile(optimizer=sgd, loss='categorical_crossentropy')

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           zeropadding2d_input_1[0][0]      
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 64, 224, 224)  36928       zeropadding2d_2[0][0]            
___________________________________________________________________________________________

In [11]:
thumbs = []
thumbs.extend(thumb_red_squares)
thumbs.extend(thumb_green_squares)
thumbs.extend(thumb_blue_squares)
thumbs.extend(thumb_red_circles)
thumbs.extend(thumb_green_circles)
thumbs.extend(thumb_blue_circles)

items = np.arange(len(vgg_labels))
np.random.shuffle(items)
thumb_images = []
representations = []
for i in items[0:200]:
    representations.append(vgg.get_representation(vgg_shapes[i])[0])
    thumb_images.append(thumbs[i])


In [12]:
%matplotlib 
t0 = time()

print("Computing t-SNE embedding")
tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
X_tsne = tsne.fit_transform(representations)
plot_embedding(X_tsne, thumb_images,
                   "t-SNE embedding of the vgg flatend layer (time %.2fs)" %
                   (time() - t0))
plt.show()

Using matplotlib backend: MacOSX
Computing t-SNE embedding


In [13]:
import theano
import theano.tensor as T

from theanolm.matrixfunctions import orthogonal_weight
from lasagne.updates import sgd, apply_momentum

class ImageCaptionGenerator(object):
    
    def __init__(self, 
                 image_rep_dim, 
                 lstm_input_dim, 
                 lstm_hidden_dim, 
                 output_dim, 
                 learning_rate,vocab_size):
        self.image_rep_dim = image_rep_dim
        self.lstm_input_dim = lstm_input_dim
        self.lstm_hidden_dim = lstm_hidden_dim
        self.output_dim = output_dim
        self.learning_rate = learning_rate
        self.vocab_size = vocab_size
        self.INIT_WORD = np.asarray(np.eye(vocab_size)[0],dtype="float32")
        self.init_params()

        

    def init_params(self):
        WordEmbedding = orthogonal_weight(self.lstm_input_dim / 2, self.output_dim,scale=0.01)
        ImageEmbedding = orthogonal_weight(self.lstm_input_dim / 2, self.image_rep_dim,scale=0.01)

        self.WordEmbedding = theano.shared(value=WordEmbedding, name="WordEmbedding", borrow="True")
        self.ImageEmbedding = theano.shared(value=ImageEmbedding, name="ImageEmbedding", borrow="True")

        U_input = orthogonal_weight(self.lstm_hidden_dim, self.lstm_input_dim,scale=0.01)

        U_forget = orthogonal_weight(self.lstm_hidden_dim, self.lstm_input_dim,scale=0.01)

        U_output = orthogonal_weight(self.lstm_hidden_dim, self.lstm_input_dim,scale=0.01)

        W_input = orthogonal_weight(self.lstm_hidden_dim, self.lstm_hidden_dim,scale=0.01)

        W_forget = orthogonal_weight(self.lstm_hidden_dim, self.lstm_hidden_dim,scale=0.01)

        W_output = orthogonal_weight(self.lstm_hidden_dim, self.lstm_hidden_dim,scale=0.01)

        U = orthogonal_weight(self.lstm_hidden_dim, self.lstm_input_dim,scale=0.01)

        W = orthogonal_weight(self.lstm_hidden_dim, self.lstm_hidden_dim,scale=0.01)

        self.W = theano.shared(value=W, name="W" , borrow="True")
        self.U = theano.shared(value=U, name="U" , borrow="True")

        self.W_input = theano.shared(value=W_input, name="W_input" , borrow="True")
        self.U_input = theano.shared(value=U_input, name="U_input" , borrow="True")

        self.W_output = theano.shared(value=W_output, name="W_output" , borrow="True")
        self.U_output = theano.shared(value=U_output, name="U_output" , borrow="True")

        self.W_forget = theano.shared(value=W_forget, name="W_forget" , borrow="True")
        self.U_forget = theano.shared(value=U_forget, name="U_forget" , borrow="True")

        O_w = orthogonal_weight(self.output_dim, self.lstm_hidden_dim)
        self.O_w = theano.shared(value=O_w, name="O_w" , borrow="True")

        
        bias_input = np.zeros(self.lstm_hidden_dim, dtype=theano.config.floatX)
        bias_forget = np.zeros(self.lstm_hidden_dim, dtype=theano.config.floatX)
        bias_output = np.zeros(self.lstm_hidden_dim, dtype=theano.config.floatX)
        bias = np.zeros(self.lstm_hidden_dim, dtype=theano.config.floatX)
        
        self.bias = theano.shared(value=bias, name="bias", borrow="True")
        self.bias_input = theano.shared(value=bias_input, name="bias_input", borrow="True")
        self.bias_output = theano.shared(value=bias_output, name="bias_output", borrow="True")
        self.bias_forget = theano.shared(value=bias_forget, name="bias_forget", borrow="True")

        
        self.params = [self.U_input, self.U_forget, self.U_output, self.W_input, self.W_forget, self.W_output,
                       self.U, self.W,
                       self.O_w, self.WordEmbedding, self.ImageEmbedding,
                       self.bias, self.bias_input, self.bias_output, self.bias_forget]
    
    def define_network(self):
        
        Y = T.matrix('output')
        H = T.vector('init_state')
        
        def lstm_step(prev_output,prev_state,prev_content):
            input_gate = T.nnet.hard_sigmoid(T.dot(( self.U_input),prev_output) + T.dot(self.W_input,prev_state) + self.bias_input)
            forget_gate = T.nnet.hard_sigmoid(T.dot(( self.U_forget),prev_output) + T.dot(self.W_forget,prev_state)+ self.bias_forget)
            output_gate = T.nnet.hard_sigmoid(T.dot((self.U_output),prev_output) + T.dot(self.W_output,prev_state)+ self.bias_output)



            stabilized_input = T.tanh(T.dot((self.U),prev_output) + T.dot(self.W,prev_state) + self.bias)
            c = forget_gate * prev_content + input_gate * stabilized_input
            s = output_gate * T.tanh(c) 
            
            output = T.nnet.softmax(T.dot(self.O_w,s))[0]
            
            next_input = T.concatenate([T.dot(self.ImageEmbedding,H),T.dot(self.WordEmbedding,output)],axis=0)
            
            return [output,next_input,s,c],  theano.scan_module.until(np.argmax(output) > (self.vocab_size - 2))
        
           
        [self.output,_,self.hidden_state, self.memory_content], updates = theano.scan(
            lstm_step,
            truncate_gradient=-1,
            n_steps= 2,
            outputs_info=[None,dict(initial = T.concatenate([T.dot(self.ImageEmbedding,H), T.dot(self.WordEmbedding,self.INIT_WORD)],axis=0)),
                          dict(initial= T.zeros(self.lstm_hidden_dim, dtype=theano.config.floatX)),
                          dict(initial=T.zeros(self.lstm_hidden_dim, dtype=theano.config.floatX)),
                          ])
                                               
        
        self.predict = theano.function([H],[self.output,self.hidden_state,self.memory_content])

        params = self.params

        padded_output = T.zeros(Y.shape) #+ [np.eye(self.output_dim,dtype="float32")[self.output_dim - 1] for i in np.arange(output.shape[0],Y.shape[0])]
        padded_output = T.set_subtensor(padded_output[0:self.output.shape[0],:],self.output)
        padded_output = T.set_subtensor(padded_output[self.output.shape[0]:, :], T.eye(self.output_dim)[self.output_dim - 1])
        cost = T.sum(T.nnet.categorical_crossentropy(padded_output + 0.000001,Y))
        updates_sgd = sgd(cost, params, learning_rate=self.learning_rate)
        updates = apply_momentum(updates_sgd, params, momentum=0.9)
        self.backprop_update = theano.function([H, Y], [cost], updates=updates)


In [14]:
print(representations[0].shape[0])

cp = ImageCaptionGenerator(image_rep_dim = representations[0].shape[0], 
                 lstm_input_dim = 128, 
                 lstm_hidden_dim = 64, 
                 output_dim = len(VOCAB), 
                 learning_rate = 0.0001,
                 vocab_size = len(VOCAB))
H = T.vector('init_state')
b = theano.function([H],[T.concatenate([T.dot(cp.ImageEmbedding,H),T.dot(cp.WordEmbedding,cp.INIT_WORD)], axis=0)])
print(b(representations[0])[0].dtype)
cp.define_network()

4096


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/theanolm/matrixfunctions.py:23: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  result = numpy.random.randn(*shape)


float32


In [15]:
predicted_label = cp.predict(representations[0])
print(get_string(predicted_label[0]))

$


In [18]:

items = np.arange(len(labels))
np.random.shuffle(items)
for i in items[0:1000]:
    rep = vgg.get_representation(shapes[i])[0]
    predicted_label,_,_ = cp.predict(rep)
    str1 = get_string(predicted_label)
    str2 = get_string(labels[i])
    print(str1 == str2)
    print("Predicted: "+str1+" Taget:"+str2)
    cp.backprop_update(rep, np.asarray(labels[i],dtype="float32"))

True
Predicted: red square Taget:red square
True
Predicted: green circle Taget:green circle
True
Predicted: green circle Taget:green circle
True
Predicted: green circle Taget:green circle
True
Predicted: red circle Taget:red circle
True
Predicted: green square Taget:green square
True
Predicted: red square Taget:red square
True
Predicted: red circle Taget:red circle
True
Predicted: blue square Taget:blue square
True
Predicted: green square Taget:green square
True
Predicted: red circle Taget:red circle
True
Predicted: red circle Taget:red circle
True
Predicted: green circle Taget:green circle
True
Predicted: green circle Taget:green circle
True
Predicted: green square Taget:green square
False
Predicted: green square Taget:blue square
True
Predicted: green square Taget:green square
True
Predicted: green square Taget:green square
True
Predicted: green circle Taget:green circle
True
Predicted: green circle Taget:green circle
True
Predicted: blue square Taget:blue square
True
Predicted: red 

In [17]:
for i in np.arange(len(blue_circles)):
    rep = vgg.get_representation(blue_circles[i])[0]
    predicted_label =cp.predict(rep)
    print(get_string(predicted_label[0]))

green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle
green circle

KeyboardInterrupt: 

In [ ]:
len(predicted_label)

In [40]:
predicted_label[0].shape

(2, 7)

In [23]:
items = np.arange(len(labels))
np.random.shuffle(items)

vis = []
thumb_images = []
j = 0
for i in items[0:200]:
    j += 1
    print(j)
    rep = vgg.get_representation(vgg_shapes[i])[0]
    [predicted_label,hid,mid] = cp.predict(rep)
    vis.append(hid[-1])
    thumb_images.append(thumbs[i])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200


In [24]:
t0 = time()

print(vis[0])
print("Computing t-SNE embedding")
tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
X_tsne = tsne.fit_transform(vis)
plot_embedding(X_tsne, thumb_images,
                   "t-SNE embedding of LSTM last hidden state (time %.2fs)" %
                   (time() - t0))
plt.show()

[ 0.38963675  0.60690409 -0.22027522 -0.27852425 -0.52409965  0.06685238
  0.17330153 -0.55358469 -0.34003252 -0.19725627  0.01612551  0.03730985
 -0.11937071  0.34422746 -0.63399881 -0.73023814  0.20308191  0.43470782
  0.68996763  0.57354277  0.20146498 -0.49489373  0.57039899 -0.536879
 -0.45516345 -0.71020764  0.26844159 -0.36713663  0.29456237  0.0801912
  0.13269234 -0.450663   -0.52784783 -0.78659594  0.51392889  0.4220275
 -0.63478076 -0.84359455 -0.80809492  0.56679338 -0.88101953  0.42500702
 -0.52293795  0.87861067 -0.25174874 -0.23775513  0.46387538  0.0936369
 -0.1426889   0.74599594 -0.29778925 -0.5753696  -0.25984749 -0.09595632
 -0.39324856  0.6865145   0.48474559  0.27850679 -0.30938685 -0.50200725
 -0.77971679 -0.34448048  0.33501869 -0.33681682]
Computing t-SNE embedding
